In [1]:
from pprint import pprint
import sqlite3

In [2]:
DB_FILE = "images.db"
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = [row[0] for row in cursor.fetchall()]
pprint(tables)

In [ ]:
cursor.execute("SELECT * FROM themes")
pprint(cursor.fetchall())

In [ ]:
cursor.execute("PRAGMA table_info(images)")
pprint(cursor.fetchall())

In [ ]:
def update_missing_previews():
    import random  # in case not already imported
    # Update collections missing preview_image
    cursor.execute("SELECT id, name, theme FROM collections")
    collections_missing = cursor.fetchall()
    for col in collections_missing:
        col_id, collection_name, theme = col
        cursor.execute("SELECT preview_image FROM images WHERE collection=? AND theme=? AND preview_image IS NOT NULL", 
                       (collection_name, theme))
        previews = cursor.fetchall()
        if previews:
            chosen_preview = random.choice([p[0] for p in previews])
            cursor.execute("UPDATE collections SET preview_image=? WHERE id=?", (chosen_preview, col_id))
            conn.commit()
            print(f"Updated collection {collection_name} with preview {chosen_preview}")

    # Update themes missing preview_image
    cursor.execute("SELECT id, name FROM themes")
    themes_missing = cursor.fetchall()
    for theme_row in themes_missing:
        theme_id, theme_name = theme_row
        cursor.execute("SELECT preview_image FROM collections WHERE theme=? AND preview_image IS NOT NULL", (theme_name,))
        previews = cursor.fetchall()
        if previews:
            chosen_preview = random.choice([p[0] for p in previews])
            cursor.execute("UPDATE themes SET preview_image=? WHERE id=?", (chosen_preview, theme_id))
            conn.commit()
            print(f"Updated theme {theme_name} with preview {chosen_preview}")

update_missing_previews()

In [ ]:
import boto3

# MinIO Configuration
MINIO_ENDPOINT = "http://localhost:9500"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"
MINIO_BUCKET = "photo-gallery"

s3_client = boto3.client(
    "s3", 
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

key = "photo-gallery/无限暖暖/游戏内拍摄/2024_12_05_22_01_46_1533941.webp"
s3_client.get_object(Bucket=MINIO_BUCKET, Key=key)

import io
from PIL import Image

response = s3_client.get_object(Bucket=MINIO_BUCKET, Key=key)
image = Image.open(io.BytesIO(response["Body"].read()))
image.show()